In [23]:
from datetime import datetime
import pandas as pd
import numpy as np
from dateutil.parser import parse
import datetime
from dateutil.parser import parse
import math
from numpy import mean

from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_validate, StratifiedKFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline as SKLpipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from dtreeviz.trees import dtreeviz 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as IMBLpipeline

from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 35)

In [24]:
# read df pickle
df_alg = pd.read_pickle("objects/df_alg-HAB_preprocessing_5_1")
# data = pd.read_pickle("data/preprocessed/hab_org-data-HAB_part2-preprocessing-5_2")
data = pd.read_pickle("data/preprocessed/hab_interp_data-HAB_part2-preprocessing-5_2")

data.drop(columns=["sampling station", "date"], inplace=True)
# data.set_index('date', inplace=True)


# slice by station and time
# data = data[data["sampling station"] == "Debeli_rtic"].loc["2008-01-01" : "2021-12-31"]

data.isnull().sum()

DSP                        1
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    422
salinity                  21
T                         59
SECCHI                   450
DIN                      352
PO4-P                    349
Soca                       0
month                      0
lipophylic_toxins        320
dtype: int64

In [25]:
# Class distribution
data["lipophylic_toxins"].value_counts(dropna=False)

neg    996
NaN    320
poz    136
Name: lipophylic_toxins, dtype: int64

In [26]:
# move month to first place
cols = data.columns.tolist()  # Get a list of column names
cols = [cols[-2]] + cols[:-2] + [cols[-1]]  # Move the one before the last column to the first position
data = data[cols]

In [27]:
# data.drop(columns=["Chl-a","PO4-P","DIN","SECCHI"], inplace=True)
data.drop(columns=["SECCHI",  ], inplace=True)#,"Chl-a", "PO4-P", "DIN",
data.isnull().sum()

month                      0
DSP                        1
Dinophysis caudata         1
Dinophysis fortii          1
Phalacroma rotundatum      1
Dinophysis sacculus        1
Dinophysis tripos          1
sun [h]                    0
air temp                   0
wind strength              0
precipitation              0
Chl-a                    422
salinity                  21
T                         59
DIN                      352
PO4-P                    349
Soca                       0
lipophylic_toxins        320
dtype: int64

# Descriptive analysis

In [28]:
# describe
description = data.describe(include='all').round(2)

# Calculate the number of missing values for each column
missing_values = data.isna().sum()
missing_values.name = 'missing_values'

# Append the missing_values row to the description DataFrame
description_with_missing = description.append(missing_values)
description_with_missing = description_with_missing.drop(['unique', 'top', 'freq'])

description_with_missing

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,month,DSP,Dinophysis caudata,Dinophysis fortii,Phalacroma rotundatum,Dinophysis sacculus,Dinophysis tripos,sun [h],air temp,wind strength,precipitation,Chl-a,salinity,T,DIN,PO4-P,Soca,lipophylic_toxins
count,1452.00,1451.00,1451.00,1451.00,1451.00,1451.00,1451.00,1452.00,1452.00,1452.00,1452.00,1030.00,1431.00,1393.00,1100.00,1103.00,1452.00,1132
mean,7.38,105.75,24.92,26.15,14.20,28.55,4.50,158.80,16.60,2.99,57.58,0.85,37.08,19.46,3.29,0.06,3294.64,NaN
std,2.82,339.89,87.92,160.06,29.93,177.90,41.18,60.63,6.06,0.41,50.71,0.71,25.50,5.33,3.94,0.16,2329.62,NaN
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,22.80,-0.82,1.48,0.00,0.09,24.13,6.24,0.06,0.00,593.92,NaN
25%,5.00,10.00,0.00,0.00,0.00,0.00,0.00,112.60,12.18,2.74,19.68,0.39,35.85,15.50,0.87,0.02,1636.00,NaN
50%,8.00,37.00,0.00,0.00,10.00,0.00,0.00,161.40,17.44,2.98,44.70,0.68,36.89,20.47,1.98,0.04,2580.82,NaN
75%,10.00,90.00,13.00,10.00,20.00,10.00,0.00,207.30,21.74,3.21,79.00,1.09,37.48,23.91,3.96,0.08,4236.82,NaN
max,12.00,7630.00,1309.00,4624.00,393.00,4639.00,1139.00,277.80,26.57,5.26,267.70,9.25,999.00,28.37,35.47,3.54,16039.87,NaN
missing_values,0.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,422.00,21.00,59.00,352.00,349.00,0.00,320


## Descriptive analysis by month

In [29]:
# table of mean values for each feature by month 
import calendar

grouped_means = data.groupby('month').mean()

# Count binary values for the categorical feature grouped by month
binary_counts = data.groupby('month')['lipophylic_toxins'].value_counts().unstack()

# Calculate the ratio of positive values for each month
sum_positive = binary_counts["poz"].sum()
positive_ratios = [i for i in (binary_counts["poz"] / sum_positive)] 
positive_ratios = [round(v * 100, 1) for v in positive_ratios]

# Change month names
month_names = {i: calendar.month_name[i] for i in range(1, 13)}

# Update the index using the month_names dictionary
grouped_means.index = grouped_means.index.map(month_names)
binary_counts.index = binary_counts.index.map(month_names)

# Concatenate the grouped_means and binary_counts DataFrames
result = pd.concat([grouped_means, binary_counts], axis=1).round(2)

# Add the positive_ratios Series as a new column to the result DataFrame
result["poz %"] = positive_ratios

result

,DSP,Dinophysis caudata,Dinophysis fortii,Phalacroma rotundatum,Dinophysis sacculus,Dinophysis tripos,sun [h],air temp,wind strength,precipitation,Chl-a,salinity,T,DIN,PO4-P,Soca,neg,poz,poz %
month,,,,,,,,,,,,,,,,,,,
January,9.00,1.16,2.96,3.66,0.09,0.20,72.62,4.95,2.71,46.53,0.76,36.95,11.07,6.79,0.07,4652.16,15,3,2.2
February,12.86,1.02,4.08,3.38,0.20,0.69,80.19,5.52,3.06,60.66,0.77,37.02,10.22,6.66,0.08,4378.79,21,3,2.2
March,12.38,0.21,0.68,3.79,0.43,0.64,118.56,7.55,3.30,30.01,0.77,37.17,9.49,4.37,0.05,3257.07,28,2,1.5
April,15.49,0.97,0.48,5.24,4.51,0.00,144.63,11.69,3.17,54.90,0.68,36.93,11.68,4.68,0.06,3826.10,66,2,1.5
May,59.00,6.00,0.97,9.67,34.25,0.15,171.96,16.22,2.89,48.55,0.96,36.57,14.87,3.92,0.07,3864.97,89,14,10.3
June,146.89,29.23,1.49,12.87,93.28,0.09,202.85,20.51,2.83,45.16,1.09,35.60,20.28,3.58,0.13,3436.93,115,12,8.8
July,237.04,67.07,44.50,19.57,97.25,0.04,227.47,23.31,2.88,38.27,0.75,35.40,23.48,3.38,0.05,2473.18,125,18,13.2
August,129.67,55.46,24.23,19.01,22.96,0.08,220.46,23.46,3.03,54.38,0.63,41.92,24.93,2.03,0.05,1637.28,126,11,8.1
September,116.79,28.04,58.58,12.81,3.57,6.04,174.65,20.06,3.11,69.73,0.64,36.59,24.38,2.06,0.05,2102.01,143,31,22.8


# Scikit-learn Analysis

# Data preprocessing for modelling

### Removing instances with unlabeled target, label encoding, train-test split

In [30]:
# Prepare for ML in scikit-learn
# labeled and unlabeled part
data_l = data[data['lipophylic_toxins'].notnull()]
data_ul = data[data['lipophylic_toxins'].isnull()]

# Remove missing values
data_l = data_l.dropna(how="any")
print(f"class distribution:")
print(data_l["lipophylic_toxins"].value_counts(dropna=False))

X = data_l.drop("lipophylic_toxins", axis=1)
y = data_l["lipophylic_toxins"]

# sklearn lable encoding
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
print(f"class encoding: ['neg','poz'] -> {le.transform(['neg','poz'])}")

class distribution:
neg    662
poz     88
Name: lipophylic_toxins, dtype: int64
class encoding: ['neg','poz'] -> [0 1]


In [31]:
# train test split
X, X_eval, y, y_eval = train_test_split(X, y, shuffle=True, stratify=y, test_size=0.30, random_state=42)

### Clean instances close to the decision boundary

Clean the dataset by removing samples close to the decision boundary. Because the dataset is heavily imbalanced in favor of clas 0 (neg) we will remove instances from this class whenever finding samples which do not agree “enough” with their neighboorhood. The EditedNearestNeighbours will be used. One other option is to use Tomek links but it is more conservative and was found to perform slightly worse.

In [32]:
from collections import Counter
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours

print(f'Original dataset shape: {Counter(y)}')
usmp = EditedNearestNeighbours()
lastMajorityCount = Counter(y)[0]
for i in range(10):
    X_res, y_res = usmp.fit_resample(X, y)
    if Counter(y_res)[0] == lastMajorityCount:
        print('Cannot remove any more samples')
        break
    else:
        print(f'Resampled dataset shape {Counter(y_res)}')
        lastMajorityCount = Counter(y_res)[0]
    X = X_res
    y = y_res

Original dataset shape: Counter({0: 463, 1: 62})
Resampled dataset shape Counter({0: 372, 1: 62})
Resampled dataset shape Counter({0: 341, 1: 62})
Resampled dataset shape Counter({0: 331, 1: 62})
Resampled dataset shape Counter({0: 327, 1: 62})
Cannot remove any more samples


# Correlation analysis on training data

In [33]:
# Calculate Pearson correlation between numeric features and binary target variable
pearson_correlations = X.corrwith(pd.Series(y), method='pearson')
pearson_correlations.name = 'Pearson'

# Calculate Spearman rank correlation between numeric features and binary target variable
spearman_correlations = X.corrwith(pd.Series(y), method='spearman')
spearman_correlations.name = 'Spearman'

# Combine the correlation values into a single dataframe
corr_df = pd.concat([pearson_correlations, spearman_correlations], axis=1)

# Create a new column for absolute Spearman correlation values
corr_df['Spearman_abs'] = corr_df['Spearman'].abs()

# Sort the dataframe by absolute Spearman correlation values
corr_df_sorted = corr_df.sort_values(by=['Spearman_abs'], ascending=False)

# Drop the absolute Spearman correlation column and return the sorted dataframe
corr_df_ranked = corr_df_sorted.drop(columns=['Spearman_abs'])

print(corr_df_ranked)

                        Pearson  Spearman
Dinophysis fortii      0.244960  0.320084
Dinophysis tripos      0.296586  0.228563
DSP                    0.151711  0.169537
salinity              -0.127015 -0.156735
precipitation          0.174680  0.135315
sun [h]               -0.149447 -0.134592
Chl-a                  0.142844  0.115245
wind strength         -0.134494 -0.110921
Soca                   0.094788  0.100006
DIN                    0.137654  0.097911
PO4-P                 -0.002119  0.095989
Dinophysis caudata     0.011765  0.078476
air temp              -0.056955 -0.078116
month                  0.069503  0.077731
Dinophysis sacculus    0.098827 -0.032319
T                     -0.005452 -0.016011
Phalacroma rotundatum -0.047214 -0.014961


## Logistic regression and p-value

We use the coef_pval() method to calculate the p-values for each coefficient in the logistic regression model, and create a dataframe with the logistic regression coefficients and corresponding p-values. Finally, we rank the features by their absolute logistic regression coefficients and print the resulting dataframe.

In [34]:
# logistic regression and p-value
import statsmodels.api as sm

# Fit logistic regression model
logit_model = sm.Logit(y, sm.add_constant(X))
result = logit_model.fit()

# Calculate p-values for the logistic regression coefficients
p_values = result.pvalues[1:]

# Create a dataframe with the logistic regression coefficients and corresponding p-values
coef_df = pd.DataFrame({'Coefficient': result.params[1:], 'P-value': p_values})

# Add feature names as the index
coef_df.index = X.columns

# Rank the features by absolute logistic regression coefficients
coef_df['Absolute Coefficient'] = coef_df['Coefficient'].abs()
coef_df_sorted = coef_df.sort_values(by=['Absolute Coefficient'], ascending=False)
coef_df_ranked = coef_df_sorted.drop(columns=['Absolute Coefficient'])

print(coef_df_ranked.round(3))

Optimization terminated successfully.
         Current function value: 0.327551
         Iterations 8
                       Coefficient  P-value
wind strength               -0.740    0.121
Chl-a                        0.465    0.170
PO4-P                       -0.449    0.528
salinity                    -0.236    0.027
air temp                     0.104    0.266
month                       -0.088    0.378
T                           -0.055    0.502
Dinophysis tripos            0.047    0.033
DIN                          0.039    0.397
Dinophysis fortii            0.031    0.039
sun [h]                     -0.013    0.099
Dinophysis sacculus          0.011    0.388
Phalacroma rotundatum       -0.009    0.638
DSP                         -0.008    0.545
precipitation                0.007    0.074
Dinophysis caudata          -0.001    0.922
Soca                        -0.000    0.033


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Fit logistic regression model using scikit-learn
logreg = LogisticRegression()
logreg.fit(X, y)

# Make predictions on the test set
y_pred = logreg.predict(X_eval)

# Calculate precision, recall, and F1-score
report = classification_report(y_eval, y_pred, target_names=['neg', 'pos'], output_dict=True)
precision = report['pos']['precision']
recall = report['pos']['recall']
f1_score = report['pos']['f1-score']

# Print results
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1_score)

Precision: 0.3888888888888889
Recall: 0.2692307692307692
F1-score: 0.3181818181818182


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


# Model Training and Evaluation

## Decision Tree Model (sklearn)

In [15]:
pd.set_option("display.max_rows", None)

pipeline = IMBLpipeline([
    ('smt', SMOTE()), 
    ('under', RandomUnderSampler()), 
    ('clf', DecisionTreeClassifier())
])

parameters = {
            'clf__max_depth': [2,3,4],
            'clf__criterion': ['gini', 'entropy', 'log_loss'],
               'clf__class_weight': ['balanced', None],
               'smt__sampling_strategy': [ 0.2, 0.3, 0.4],
               'under__sampling_strategy': [0.5, 0.6, 0.7],
               'smt__k_neighbors': [1, 3, 5]
             }
nfolds = 5
scores = ['recall', 'precision', 'f1', 'roc_auc', 'recall_weighted']
gscv_dt = GridSearchCV(pipeline, 
                    parameters, 
                    scoring=scores,
                    cv=StratifiedKFold(n_splits=nfolds, shuffle=True),
                    return_train_score=False, 
                    verbose=1, 
                    refit="f1",
                    n_jobs=-1)
resultsGSCV = gscv_dt.fit(X, y)
results = pd.DataFrame(resultsGSCV.cv_results_)
display(results.sort_values(by=[f'rank_test_recall']).transpose())

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


,177,166,155,192,182,131,178,95,103,50,98,132,196,102,48,188,107,...,477,354,258,424,270,243,466,450,324,337,334,328,408,414,451,405,342
mean_fit_time,0.020202,0.021855,0.019744,0.022603,0.019399,0.025297,0.020987,0.024459,0.020785,0.02133,0.021187,0.021624,0.02381,0.019863,0.0199,0.019429,0.021572,...,0.021302,0.019271,0.020801,0.018158,0.021525,0.019904,0.021179,0.020673,0.022508,0.02004,0.020383,0.024786,0.018707,0.018866,0.020785,0.019884,0.020461
std_fit_time,0.001659,0.005072,0.000923,0.001293,0.001055,0.003108,0.001248,0.001934,0.002184,0.002067,0.001263,0.00259,0.003543,0.000887,0.000558,0.001208,0.002397,...,0.002445,0.000769,0.001404,0.000837,0.001775,0.000835,0.000823,0.001262,0.00307,0.00085,0.001807,0.003139,0.000335,0.001262,0.002053,0.001598,0.001072
mean_score_time,0.011657,0.012033,0.011836,0.013328,0.011756,0.01356,0.012843,0.011331,0.012837,0.012092,0.012186,0.011385,0.01354,0.010966,0.011344,0.01203,0.012599,...,0.01036,0.011607,0.011704,0.010921,0.014489,0.013496,0.01233,0.012197,0.0117,0.012365,0.013139,0.013501,0.011278,0.010962,0.013325,0.011858,0.012075
std_score_time,0.000681,0.001374,0.000815,0.003543,0.001113,0.001015,0.001589,0.000639,0.001463,0.001849,0.001214,0.000508,0.002495,0.000777,0.000586,0.001257,0.00126,...,0.00148,0.000661,0.001036,0.000181,0.00333,0.002398,0.001211,0.000529,0.000633,0.002675,0.002367,0.001256,0.000855,0.000161,0.002814,0.001738,0.001428
param_clf__class_weight,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,balanced,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
param_clf__criterion,log_loss,log_loss,entropy,log_loss,log_loss,entropy,log_loss,entropy,entropy,gini,entropy,entropy,log_loss,entropy,gini,log_loss,entropy,...,log_loss,entropy,gini,log_loss,gini,gini,log_loss,log_loss,entropy,entropy,entropy,entropy,log_loss,log_loss,log_loss,log_loss,entropy
param_clf__max_depth,2,2,4,3,2,3,2,2,2,3,2,3,3,2,3,2,2,...,4,3,2,2,3,2,4,3,2,2,2,2,2,2,3,2,2
param_smt__k_neighbors,3,1,5,1,5,5,3,3,5,5,3,5,1,5,5,5,5,...,5,1,3,5,1,1,1,5,1,3,3,1,1,3,5,1,5
param_smt__sampling_strategy,0.4,0.3,0.2,0.3,0.2,0.3,0.4,0.3,0.3,0.3,0.4,0.4,0.4,0.3,0.3,0.4,0.4,...,0.2,0.3,0.4,0.2,0.2,0.2,0.4,0.2,0.2,0.3,0.2,0.3,0.3,0.2,0.2,0.2,0.2
param_under__sampling_strategy,0.5,0.6,0.7,0.5,0.7,0.7,0.6,0.7,0.6,0.7,0.7,0.5,0.6,0.5,0.5,0.7,0.7,...,0.5,0.5,0.5,0.6,0.5,0.5,0.6,0.5,0.5,0.6,0.6,0.6,0.5,0.5,0.6,0.5,0.5


In [16]:
# Evaluation
from sklearn.metrics import classification_report
y_pred = gscv.best_estimator_.steps[2][1].predict(X_eval)
print(classification_report(y_eval, y_pred))

NameError: name 'gscv' is not defined

In [ ]:
clf = gscv.best_estimator_.steps[2][1]
viz = dtreeviz(clf, X, y,
                target_name="target",
                feature_names=X.columns,
                class_names=["neg", "poz"],
             fancy=False,
               scale=1.5
              )

viz

### Random Forest Model

#### Model evaluation (Random Forest)

In [ ]:
# Random forest with grid search for parameters, testing on 5-fold CV with shuffling

pipeline = IMBLpipeline([
   ('smt', SMOTE()), 
   ('under', RandomUnderSampler()), 
    ('clf', RandomForestClassifier())
])

parameters = {
              'clf__n_estimators': [100,300,500],
              'clf__criterion': ['gini', 'entropy', 'log_loss'],
              'clf__class_weight': ['balanced', 'balanced_subsample', None],
              'smt__sampling_strategy': [ 0.2, 0.3, 0.4],
              'under__sampling_strategy': [0.5, 0.6, 0.7],
              'smt__k_neighbors': [3, 5]
             }

nfolds = 5
scores = ['recall', 'precision', 'f1', 'roc_auc']
refit_score = 'f1'
gscv_rf = GridSearchCV(pipeline, 
                    parameters, 
                    scoring=scores,
                    cv=StratifiedKFold(n_splits=nfolds, shuffle=True),
                    return_train_score=False, 
                    verbose=1, 
                    refit=refit_score,
                    n_jobs=-1)
resultsGSCV = gscv_rf.fit(X, y)
results = pd.DataFrame(resultsGSCV.cv_results_)
display(results.sort_values(by=[f'rank_test_recall']).transpose())
pd.set_option("display.max_rows", None)

In [ ]:
# Evaluation RF
from sklearn.metrics import classification_report
y_pred = gscv_rf.best_estimator_.steps[2][1].predict(X_eval)
print(classification_report(y_eval, y_pred))

Plot the mean ROC curve of the algorithm with best performing parameter selection. We will perform CV once again and plot the ROC curve for each fold and compute and plot the mean.

In [ ]:
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=3, shuffle=True)
classifier = resultsGSCV.best_estimator_

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(10,8))
for i, (train, test) in enumerate(cv.split(X_eval, y_eval)):
    classifier.fit(X_eval.iloc[train], y_eval[train])
    viz = RocCurveDisplay.from_estimator(
        classifier,
        X_eval.iloc[test],
        y_eval[test],
        name="fold {}".format(i),
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Baseline (random prediction)", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

clfname = [str(step[1].__class__.__name__) for step in classifier.steps if step[0]=='clf'][0]
ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title=f'{clfname} evaluation (ROC-AUC, {nfolds}-fold CV)',
)
ax.legend(loc="lower right")
plt.tight_layout()
plt.show()

Plot the mean precision-recall curve. The approach is the same as for the mean ROC curve.

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

cv = StratifiedKFold(n_splits=3, shuffle=True)
classifier = resultsGSCV.best_estimator_

prs = []
aucs = []
mean_r = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(10,8))
for i, (train, test) in enumerate(cv.split(X_eval, y_eval)):
    classifier.fit(X.iloc[train], y[train])
    viz = PrecisionRecallDisplay.from_estimator(
        classifier,
        X_eval.iloc[test],
        y_eval[test],
        name="fold {}".format(i),
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_pr = np.interp(mean_r, viz.recall[::-1], viz.precision[::-1])
    prs.append(interp_pr)

mean_p = np.mean(prs, axis=0)
ax.plot(
    mean_r,
    mean_p,
    color="b",
    label=f"mean",
    lw=2,
    alpha=0.8,
)
ax.legend(loc="lower left")
clfname = [str(step[1].__class__.__name__) for step in classifier.steps if step[0]=='clf'][0]
ax.set(
    # xlim=[-0.05, 1.05],
    # ylim=[-0.05, 1.05],
    title=f'{clfname} evaluation (precision-recall, {nfolds}-fold CV)')
plt.tight_layout()
plt.show()

#### Feature importance (Random Forest)

In [ ]:
# Feature importance of model (best RandomForest from gridsearch) with three methods!

fig, (ax2) = plt.subplots(1, 1, figsize=(10,9))
plt.subplots_adjust(wspace=1.1)

rf = gscv.best_estimator_.steps[2][1]

# Get feature importance with Permutation Based Feature Importance (randomly shuffles each feature and compute the 
# change in the model’s performance. The features which impact the performance the most are the most important one).
perm_importance = permutation_importance(rf, X, y)
perm_sorted_idx = perm_importance.importances_mean.argsort()
x2 = X.columns[perm_sorted_idx]
y2 = perm_importance.importances_mean[perm_sorted_idx]
ax2.barh(x2, y2)
ax2.set_title("Permutation Importance Random Forest")

In [ ]:
# probaj enako z X_eval in primerjaj

In [ ]:
# Get feature importance with SHAP
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X)
RF_shap = shap.summary_plot(shap_values, X, plot_type="bar")

In [ ]:
# probaj enako z X_eval in primerjaj

In [ ]:
# SHAP summary plot
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X)
classid = 1
shap.summary_plot(shap_values[classid], X, max_display=len(X.columns), class_names=le.classes_)

### Neural Network Model

#### Model Evaluation (MLP)

In [ ]:
# Preprocessing for NN in scikit_learn

# Model evaluation with the pipeline of SMOTE oversampling and undersampling on the training dataset only (within each cross-validation fold)!

# one-hot encoding of month feature
Xohe = pd.get_dummies(X, columns=["month"])

X_display = Xohe.copy()  # *used for SHAP visualization so we can show unscaled values

# scalling numeric values for NN
scaled_array = StandardScaler().fit_transform(Xohe)
Xsc = pd.DataFrame(scaled_array, columns=Xohe.columns)

In [ ]:
# pd.set_option("display.max_rows", None)

In [ ]:
# MLP with grid search for parameters, testing on 5-fold CV with shuffling

pipeline = IMBLpipeline([
    ('over', SMOTE()),
    ('under', RandomUnderSampler()),
    ('clf', MLPClassifier(solver='lbfgs', max_iter=5000))
])

parameters = {'over__k_neighbors': range(1,7),
              'over__sampling_strategy': [0.5, 0.6, 0.8], # probaj poveča ovresampling do 0.9
              'under__sampling_strategy': [0.6, 0.7, 0.8],
              'clf__hidden_layer_sizes': [(2, ), (2, 2), (3,), (3,3)],
             }
nfolds = 5
scores = ['recall', "precision", 'f1', 'roc_auc']
gscv_NN = GridSearchCV(pipeline, 
                    parameters, 
                    scoring=scores,
                    cv=StratifiedKFold(n_splits=nfolds, shuffle=True),
                    n_jobs= -1, 
                    return_train_score=False, 
                    verbose=1, 
                    refit= "recall")
resultsGSCV = gscv_NN.fit(Xsc, y)
results = pd.DataFrame(resultsGSCV.cv_results_)
display(results.sort_values(by=[f'rank_test_recall']).transpose())

In [ ]:
# Evaluation 
from sklearn.metrics import classification_report
X_eval_ohe = pd.get_dummies(X_eval, columns=["month"])
scaler = StandardScaler().fit(Xohe)
X_eval_sc = scaler.transform(X_eval_ohe)
X_eval_sc = pd.DataFrame(X_eval_sc, columns=Xohe.columns)
y_pred = gscv.best_estimator_.steps[2][1].predict(X_eval_sc)
print(classification_report(y_eval, y_pred))

#### Feature Importance (MLP)

In [ ]:
# Feature importance of model (MLP)  (no cross-validation!)

fig, (ax2) = plt.subplots(1, 1, figsize=(10,9))
plt.subplots_adjust(wspace=3)

MLP = gscv.best_estimator_.steps[2][1]

# Get feature importance with Permutation Based Feature Importance (randomly shuffles each feature and compute the 
# change in the model’s performance. The features which impact the performance the most are the most important one).
perm_importance = permutation_importance(MLP, Xsc, y)
perm_sorted_idx = perm_importance.importances_mean.argsort()
x2 = Xsc.columns[perm_sorted_idx]
y2 = perm_importance.importances_mean[perm_sorted_idx]
ax2.barh(x2, y2)
ax2.set_title("Permutation Importance MLP")

#### Feature importance with SHAP

In [ ]:
X = Xsc.copy()
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y, test_size=0.25)

nn = MLPClassifier(hidden_layer_sizes=(3,3), solver='lbfgs', max_iter=5000)
model = nn.fit(X.to_numpy(), y)

First, visualize the impact of all features on both classes in one chart. We are using KernelExplainer but simpler general Explainer should be also tested once the SHAP code fixes all bugs.

**Note: SHAP explanations change between runs because of sampling and probably other random factors!**

In [ ]:
# explain the model's predictions using SHAP
import shap
import warnings
warnings.filterwarnings("ignore")
shap.initjs()

explainer = shap.KernelExplainer(model.predict_proba, shap.sample(X_eval_sc,20))
shap_values = explainer.shap_values(X_eval_sc, nsamples=50)
shap.summary_plot(shap_values, X_eval_sc, max_display=len(X.columns), class_names=le.classes_)

Now for each class separately. We observe the impact of features on the returned model's probability for a given class.

In [ ]:
explainer = shap.KernelExplainer(model.predict_proba, shap.sample(Xsc,50)) #morda treba zmanjšat število, ali brez sample in samo X_eval
shap_values = explainer.shap_values(Xsc, nsamples=50)
classid = 1
shap.summary_plot(shap_values[classid], Xsc, max_display=len(X.columns), class_names=le.classes_)

In [ ]:
# Try dependence contribution plot
explainer = shap.KernelExplainer(model.predict_proba, shap.sample(X_eval_sc,50))
shap_values = explainer.shap_values(X_eval_sc, nsamples=50)
shap.dependence_plot('salinity', shap_values[1], X_eval_sc,) #interaction_index="salinity"

Example intepretation: The fact this slopes upward says the higher the soca flow, the higher the model's prediction is for poz/neg. The spread suggests that other features must interact with Soca flow. 
In general, high Soca flow increases the chance of poz/neg. But if the sea temp is moderate or low, that trend reverses and even high soca flow does not increase preditions of poz/neg as the sea temp is too low.
https://www.kaggle.com/code/dansbecker/advanced-uses-of-shap-values

Now let's explain the prediction of a single instance. We will show the explanation of the bigger predicted probability to see why the model decided as it did. But in practice we could be interested only in the explanation of the probability of the positive prediction.

In [ ]:
instanceID = 10
instance = X.iloc[[instanceID]]
display_instance = X_display.iloc[[instanceID]]

prediction = model.predict(instance)[0]
prediction_probs = model.predict_proba(instance)[0]
print(f'real value: {y[instanceID]}, \npredicted: {prediction}, \npredicted probs: {prediction_probs}')
max_p_id = prediction_probs.argmax()  # we will show the explanation of the bigger predicted probability
print(f'Explanation for prediction: class={max_p_id}, p={prediction_probs.max()}')

explainer = shap.KernelExplainer(model.predict_proba, shap.sample(X, 50))
shap_values = explainer.shap_values(instance, nsamples=500)
shap.force_plot(explainer.expected_value[max_p_id], shap_values[max_p_id], features=display_instance)

Show the mean values of features as it may help understanding this particular instance data in the plot above.

In [ ]:
data = pd.get_dummies(data, columns=["month"])


pd.DataFrame([data[data['lipophylic_toxins']=='neg'].mean(), data[data['lipophylic_toxins']=='poz'].mean()], index=['neg','pos']).T

### Conclusion

In [ ]:
# Summary table of prediction results
RF_recall = round(RF_recall_best_k[1], 2)
RF_auc = round(RF_auc_best_k[1], 2)
MLP_recall = round(MLP_recall_best_k[1], 2)
MLP_auc = round(MLP_auc_best_k[1], 2)

summary = pd.DataFrame(
    [
        (
            "RF",
            RF_recall_score,
            RF_auc_score,
        ),
        (
            "MLP",
            MLP_recall_score,
            MLP_auc_score,
        ),
        (
            "RF (smote)",
            RF_recall,
            RF_auc,
        ),
        (
            "MLP (smote)",
            MLP_recall,
            MLP_auc,
        ),
        (
            "Decision tree (J48)*",
            0.56,
            0.18,
        ),
    ],
    columns=("Model", "Recall", "ROC AUC"),
).set_index("Model")

print("Table summarising the prediction results of the used classifiers, both with and without SMOTE resampling:\n")
summary.round(2)

As can be seen resampling with SMOTE helped to improve the results substantially, especially when calculating recall. The highest recall and ROC AUC was achieved with Random Forest with the re-sampled data. Both recall and ROC AUC suggest Random Forest as beeing the better classifier for this particular problem. Recall is a crucial metric as it gives indication of what fraction of true positive instances have been predicted. Since the models predict toxins in seashells (food) it is crucial that as few positives as possible are missed.

Due to the use of SMOTE resampling (upsampling and downsampling) in combinaiton with cross-validation it was curcial to do the resampling within each fold to avoid data lekeage and validate on original (unsampled) data. In addition, I have optimised the model with regard to the k-values of SMOTE, all of which brought along some complexity. So for the parameter tuning of Random Forest and MLP various parameter settings have been tried  and the model with best performing settings has been chosen.

The decision tree J48 algorithm was run within Weka on a slightly different dataset (missing values were not removed to use as many instances as possible, cross validation was 10-fold as opposed to 3-fold due to a higher dataset etc.) thus this results are not directly comparable but were provided as a reference to give an indication of the performance of this algorithm. 

As can be seen in the feature importance bar plots above, similar features were on the top despite using two different classification algorithms and two different feature ranking methods. If we consider just the three highest-ranking features of each of the feature ranking methods for both algortihms (RF and MLP) the features that overlap are DSP, DSP_like, ASP, Dinophysis fortii and Dinophysis caudata. These can be shown to the domain experts for validation and interpretation.